# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
# ORIGINAL
import torch
import torch.nn as nn
import torchvision.models as models


class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        #resnet = models.resnet50(pretrained=True)
        resnet = models.resnet101(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad_(False)
        
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.embed = nn.Linear(resnet.fc.in_features, embed_size)

    def forward(self, images):
        features = self.resnet(images)
        features = features.view(features.size(0), -1)
        features = self.embed(features)
        return features

In [2]:
# ATTENTION
class BahdanauAttention(nn.Module):
    """
    Bahdanau Attention.
    Reference:
    https://blog.floydhub.com/attention-mechanism/#bahdanau-att-step1 --> Attention Mechanism
    https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning --> PyTorch Image Captioning
    """

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(BahdanauAttention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att = nn.Linear(attention_dim, 1)  # linear layer to calculate values to be softmax-ed
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, decoder_dim)
        :return: attention weighted encoding, weights
        """
        att1 = self.encoder_att(encoder_out)  # (batch_size, attention_dim)
        #print('encoder_out: ', encoder_out.shape)
        #print('att1 -> (batch_size, attention_dim): ', att1.shape)
        att2 = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        #print('decoder_hidden: ', decoder_hidden.shape)
        #print('att2 -> (batch_size, attention_dim): ', att2.shape)
        #att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        att = self.full_att(self.tanh(att1 + att2))  # (batch_size, num_pixels)
        #print('att -> (batch_size, num_pixels): ', att.shape)
        alpha = self.softmax(att)  # (batch_size, num_pixels)
        #print('alpha -> (batch_size, num_pixels): ', alpha.shape)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_dim)
        #print('attention_weighted_encoding -> (batch_size, encoder_dim): ', attention_weighted_encoding.shape)

        return attention_weighted_encoding, alpha

    
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, decoder_size, vocab_size, attention_size=512, encoder_size=512, dropout=0.25):
        '''
        [See the diagram of the decoder in Notebook 1]
        The RNN needs to have 4 basic components :
            1. Word Embedding layer : maps the captions to embedded word vector of embed_size.
            2. LSTM layer : inputs( embedded feature vector from CNN , embedded word vector ).
            3. Hidden layer : Takes LSTM output as input and maps it 
                          to (batch_size, caption length, decoder_size) tensor.
            4. Linear layer : Maps the hidden layer output to the number of words
                          we want as output, vocab_size.
        '''
        
        super(DecoderRNN, self).__init__()
        self.encoder_size = encoder_size
        self.attention_size = attention_size
        self.embed_size = embed_size
        self.decoder_size = decoder_size
        self.vocab_size = vocab_size
        self.dropout = dropout
        
        '''
        Embedding layer parameters:
            vocab_size : size of the dictionary of embeddings, 
                basically the number of tokens in the vocabulary(word2idx) for that batch of data.
            embed_size : the size of each embedding vector of captions
        '''
        self.embedding_layer = nn.Embedding(vocab_size, embed_size)

        '''
        BahdanauAttention layer parameters:
        '''
        self.attention_layer = BahdanauAttention(encoder_size, decoder_size, attention_size)  # attention network
        
        '''
        LSTM layer parameters :
            input_size  = embed_size 
            hidden_size = decoder_size     # number of units in hidden layer of LSTM  
            num_layers  = 1               # number of LSTM layers ( = 1, by default )
            batch_first = True            # input , output need to have batch size as 1st dimension
            dropout     = 0               # did not use dropout 
        
        Other parameters were not changed from default values provided in the PyTorch implementation.
        '''
        #print('embed_size: ', embed_size)
        #print('encoder_size: ', encoder_size)
        #print('decoder_size: ', decoder_size)
        
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html
        self.lstm_layer = nn.LSTMCell( input_size = embed_size+encoder_size, 
                             hidden_size = decoder_size, 
                             bias=True)
                             #num_layers = num_layers, 
                             #dropout = dropout, 
                             #batch_first=True )

        self.init_h_layer = nn.Linear(encoder_size, decoder_size)  # linear layer to find initial hidden state of LSTMCell
        self.init_c_layer = nn.Linear(encoder_size, decoder_size)  # linear layer to find initial cell state of LSTMCell
        self.linear_fc_layer = nn.Linear(decoder_size, vocab_size)  # linear layer to find scores over vocabulary
        self.init_weights()  # initialize some layers with the uniform distribution

    def init_weights(self):
        """
        Initializes some parameters with values from the uniform distribution, for easier convergence.
        """
        self.embedding_layer.weight.data.uniform_(-0.1, 0.1)
        self.linear_fc_layer.bias.data.fill_(0)
        self.linear_fc_layer.weight.data.uniform_(-0.1, 0.1)


    def init_decoder_state(self, encoder_out):
        """
        Creates the initial hidden and cell states for the decoder's LSTM based on the encoded images.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, encoder_dim)
        :return: hidden state, cell state
        """
        #print('encoder_out: ', encoder_out)
        #print('encoder_out.shape: ', encoder_out.shape)
        #mean_encoder_out = encoder_out.mean(dim=0) # TODO: DA SISTEMARE
        #print('mean_encoder_out: ', mean_encoder_out)
        #print('mean_encoder_out.shape: ', mean_encoder_out.shape)
        #h = self.init_h_layer(mean_encoder_out)  # (batch_size, decoder_dim)
        #c = self.init_c_layer(mean_encoder_out)
        h = self.init_h_layer(encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c_layer(encoder_out)
        return h, c

    
    def forward(self, encoder_out, captions):
        '''
        Arguments :
        For a forward pass, the instantiation of the RNNDecoder class
        receives as inputs 2 arguments  :
        -> encoder_out : ouput of CNNEncoder having shape (batch_size, embed_size).
        -> captions : a PyTorch tensor corresponding to the last batch of captions 
                      having shape (batch_size, caption_length) .
        NOTE : Input parameters have first dimension as batch_size.
        '''
        
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size
        #print ('batch_size: ', batch_size)
        #print ('encoder_out.shape: ', encoder_out.shape)
        #print ('encoder_dim: ', encoder_dim)
        #print ('vocab_size: ', vocab_size)
        #print ('captions.shape: ', captions.shape)

        # Flatten image
        #encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        #num_pixels = encoder_out.size(1)
        #print('num_pixels: ', num_pixels)
        #print('encoder_out -> (batch_size, encoder_dim): ', encoder_out.shape)
        
        # Discard the <end> word to avoid the following error in Notebook 1 : Step 4
        # (outputs.shape[1]==captions.shape[1]) condition won't be satisfied otherwise.
        # AssertionError: The shape of the decoder output is incorrect.
        # captions.shape: torch.Size([10, 16])
        #captions = captions[:, :-1] 
        captions_length = captions.shape[1]
        captions_length_list = [captions_length for i in range(captions.shape[0])]
        caption_lengths, sort_ind = torch.FloatTensor(captions_length_list).sort(dim=0, descending=True)
        caption_lengths = (caption_lengths).tolist()
        #print('caption_lengths: ',caption_lengths)
        
        # Pass image captions through the word_embeddings layer.
        # output shape : (batch_size, caption length , embed_size)
        embeddings = self.embedding_layer(captions) # (batch_size, max_caption_length, embed_dim)
        #print ('embeddings -> (batch_size, max_caption_length, embed_dim): ', embeddings.shape)

        # Initialize LSTM state
        #h, c = self.init_decoder_state(encoder_out)  # (batch_size, decoder_dim)
        h, c = self.init_decoder_state(encoder_out)  # (batch_size, decoder_dim)
        #print('h.shape -> (batch_size, decoder_dim): ',h.shape)
        #print('c.shape -> (batch_size, decoder_dim): ',c.shape)

        # Create tensors to hold word predicion scores and alphas
        predictions = torch.zeros(batch_size, int(max(caption_lengths)), vocab_size).to(device)
        alphas = torch.zeros(batch_size, int(max(caption_lengths))).to(device)
        #print('predictions.shape: ', predictions.shape)
        #print('alphas.shape: ', alphas.shape)

        # At each time-step, decode by
        # attention-weighing the encoder's output based on the decoder's previous hidden state output
        # then generate a new word in the decoder with the previous word and the attention weighted encoding
        for t in range(int(max(caption_lengths))):
            batch_size_t = sum([l > t for l in caption_lengths])
            #print('Ciclo For nro: ', t)
            #print('batch_size_t: ', batch_size_t)
            attention_weighted_encoding, alpha = self.attention_layer(encoder_out, h)

            #gate = self.sigmoid(self.f_beta(h[:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
            #attention_weighted_encoding = gate * attention_weighted_encoding

            #print('attention_weighted_encoding.shape: ', attention_weighted_encoding.shape)
            #print('alpha.shape: ', alpha.shape)
            #print('embeddings[:, t, :].shape: ', embeddings[:, t, :].shape)
            #print('embeddings.shape: ', embeddings.shape)
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            #print('embeddings[:, t, :] + attention_weighted_encoding: ', torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).shape)
            #print('embeddings + attention_weighted_encoding: ', torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).shape)

            # TODO: problema con c, viene creato come torch.Size([10, 512]) e poi diventa tupla dopo questa chiamata 
            #x, (h, c) = self.lstm_layer(torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).unsqueeze(1), (h.unsqueeze(0), c.unsqueeze(0)))  # (batch_size_t, decoder_dim)
            h, c = self.lstm_layer(torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1), (h, c))  # (batch_size_t, decoder_dim)
            #print('x.shape: ', x.shape)
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            h = h.squeeze()
            c = c.squeeze()
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            #print('c: ', c)

            preds = self.linear_fc_layer(h)  # (batch_size_t, vocab_size)
            #print('preds.shape: ', preds.shape)
            #print('alpha.shape: ', alpha.shape)
            #preds = self.fc(self.dropout(h))  # (batch_size_t, vocab_size)
            #predictions[:batch_size_t, t, :] = preds
            #alphas[:batch_size_t, t, :] = alpha
            predictions[:, t, :] = preds
            alphas[:, t] = alpha.squeeze()
            #print('predictions.shape: ', predictions.shape)
            #print('alphas.shape: ', alphas.shape)
        
        return predictions #, captions, captions_length_list, alphas

    def sample(self, inputs, states=None, max_len=20):
        " accepts pre-processed image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "
        pass

In [8]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
#from model import EncoderCNN, DecoderRNN
#from modelAttention import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 50          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

annotations_file:  data/annotations/captions_train2014.json
vocab_from_file:  True
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.69s)
creating index...
index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [00:51<00:00, 8040.66it/s]


In [9]:
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=1)
optimizer = torch.optim.Adam(params, lr=.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

cuda


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [10]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

print(time.ctime())

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
#response = requests.request("GET", "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", headers={"Metadata-Flavor":"Google"})

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            #requests.request("POST", "https://nebula.udacity.com/api/v1/remote/keep-alive", headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        #print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
            print(time.ctime())
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
        print(time.ctime())

# Close the training log file.
f.close()

Sat Sep 19 18:52:31 2020
Epoch [1/1], Step [50/3236], Loss: 5.0726, Perplexity: 159.5962
Sat Sep 19 19:03:52 2020
Epoch [1/1], Step [100/3236], Loss: 4.8588, Perplexity: 128.8734
Sat Sep 19 19:13:18 2020
Epoch [1/1], Step [150/3236], Loss: 4.9602, Perplexity: 142.6159
Sat Sep 19 19:22:11 2020
Epoch [1/1], Step [200/3236], Loss: 4.7245, Perplexity: 112.6700
Sat Sep 19 19:30:26 2020
Epoch [1/1], Step [250/3236], Loss: 4.5574, Perplexity: 95.3391
Sat Sep 19 19:38:39 2020
Epoch [1/1], Step [300/3236], Loss: 4.6651, Perplexity: 106.1738
Sat Sep 19 19:47:40 2020
Epoch [1/1], Step [350/3236], Loss: 4.8009, Perplexity: 121.6187
Sat Sep 19 19:55:56 2020
Epoch [1/1], Step [400/3236], Loss: 4.7269, Perplexity: 112.9422
Sat Sep 19 20:03:34 2020
Epoch [1/1], Step [450/3236], Loss: 4.6359, Perplexity: 103.1223
Sat Sep 19 20:11:26 2020


KeyboardInterrupt: 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.